# PyPetaKit5D Examples

## Install PyPetaKit5D in your env

In [ ]:
# The installation may take a long time
# You can skip this step if you've already installed it in the env you are using for this notebook
%pip install --no-binary :all: --no-cache-dir PyPetaKit5D

## Download the Cell Image Dataset

In [ ]:
# You can set the folder where our demo dataset will be downloaded to
# By default it will be the Downloads folder in your home directory
# I.E. Setting petakit5d_demo_dataset_install_dir = "~/Downloads/" will install
# the dataset to ~/Downloads/PetaKit5D_demo_cell_image_dataset/
import os
import subprocess
import urllib.request

# Change this variable to your desired download location
petakit5d_demo_dataset_install_dir = "~/Downloads/"

petakit5d_demo_dataset_install_dir = os.path.expanduser(petakit5d_demo_dataset_install_dir)
petakit5d_demo_dataset_install_dir = os.path.join(petakit5d_demo_dataset_install_dir,'')
petakit5d_demo_dataset_url = "https://www.dropbox.com/scl/fi/1xr8jjvta45hqo1ejp2s6/PetaKit5D_demo_cell_image_dataset.tar?rlkey=0glipffj32l212ul7aw4rjl1y&st=1smi4rlv&dl=1"
petakit5d_demo_dataset_dir_loc = os.path.join(petakit5d_demo_dataset_install_dir, "PetaKit5D_demo_cell_image_dataset/")
petakit5d_demo_dataset_tar_loc = os.path.join(petakit5d_demo_dataset_install_dir, "PetaKit5D_demo_cell_image_dataset.tar")
os.makedirs(os.path.dirname(petakit5d_demo_dataset_install_dir), exist_ok=True)
if not os.path.exists(petakit5d_demo_dataset_dir_loc):
    print("Downloading the demo dataset...")
    urllib.request.urlretrieve(petakit5d_demo_dataset_url, petakit5d_demo_dataset_tar_loc)
    print("Demo dataset downloaded successfully!")
    print("Untarring the demo dataset...")
    process = subprocess.Popen(
    f"tar -xf \"{petakit5d_demo_dataset_tar_loc}\" -C \"{petakit5d_demo_dataset_install_dir}\"", shell=True)
    process.wait()
    print("Demo dataset untarred successfully!")
    print("Removing the tar file...")
    os.remove(petakit5d_demo_dataset_tar_loc)
    print("Finished!")
    print(f"The demo dataset is located here: {petakit5d_demo_dataset_dir_loc}")
else:
    print(f"The demo dataset is already located here: {petakit5d_demo_dataset_dir_loc}")


## Run a DSR on our demo dataset

In [ ]:
from PyPetaKit5D import XR_deskew_rotate_data_wrapper

# Set your data path to our demo dataset
dataPath = '~/Downloads/PetaKit5D_demo_cell_image_dataset/'
dataPath_exps = [dataPath]

# run deskew if true
deskew = True
# run rotation if true
rotate = True
# use combined processing if true. Note: if you only need deskew without rotation, set both Rotate and DSRCombined
# as false.
DSRCombined = True
# xy pixel size
xyPixelSize = 0.108
# z scan step size
dz = 0.3
# Skew angle
skewAngle = 32.45
# if true, objective scan; otherwise, sample scan
objectiveScan = False
# scan direction
reverse = True
# channel patterns to map the files for processing
channelPatterns = ['CamA', 'CamB']

# flat field related parameters
# disable flat field for this demo
FFCorrection = False
# lower bound cap for flat field
lowerLimit = 0.4
# constant offset after flat field correction
constOffset = 1
# flat field image paths
FFImagePaths = [
    dataPath + 'FF/averaged/ff_CamA_ch0_CAM1_stack0000_488nm_0000000msec_0096911533msecAbs_000x_000y_000z_0017t.tif',
    dataPath + 'FF/averaged/ff_CamB_ch0_CAM1_stack0000_488nm_0000000msec_0096911533msecAbs_000x_000y_000z_0017t.tif']

# background image paths
backgroundPaths = [dataPath + 'FF/KorraFusions/AVG_DF400_CamA_10ms.tif',
                   dataPath + 'FF/KorraFusions/AVG_DF400_CamB_10ms.tif']

# if true, use large scale processing pipeline (split, process, and then merge)
largeFile = False
# true if input is in zarr format
zarrFile = False
# save output as zarr if true
saveZarr = False
# block size to save the result
blockSize = [256, 256, 256]
# save output as uint16 if true
save16bit = True

# use slurm cluster if true, otherwise use the local machine (master job)
parseCluster = False
# use master job for task computing or not.
masterCompute = True
# configuration file for job submission
configFile = ''
# if true, use Matlab runtime (for the situation without matlab license)
mccMode = True

XR_deskew_rotate_data_wrapper(dataPath_exps, deskew=deskew, rotate=rotate,
                              DSRCombined=DSRCombined, xyPixelSize=xyPixelSize, dz=dz, skewAngle=skewAngle,
                              objectiveScan=objectiveScan, reverse=reverse, channelPatterns=channelPatterns,
                              FFCorrection=FFCorrection, lowerLimit=lowerLimit, constOffset=constOffset,
                              FFImagePaths=FFImagePaths, backgroundPaths=backgroundPaths, largeFile=largeFile,
                              zarrFile=zarrFile, saveZarr=saveZarr, blockSize=blockSize, save16bit=save16bit,
                              parseCluster=parseCluster, masterCompute=masterCompute, configFile=configFile,
                              mccMode=mccMode)


## Run a deconvolution on our demo dataset

In [ ]:
from PyPetaKit5D import XR_decon_data_wrapper

# Set your data path to our demo dataset
dataPath = '~/Downloads/PetaKit5D_demo_cell_image_dataset/'
dataPaths = [dataPath]
rt = dataPath

# xy pixel size in um
xyPixelSize = 0.108
# z step size
dz = 0.3
# scan direction
Reverse = True
# psf z step size (we assume xyPixelSize also apply to psf)
dzPSF = 0.3

# if true, check whether image is flipped in z using the setting files
parseSettingFile = False

# channel patterns for the channels, the channel patterns should map the
# order of PSF filenames.
ChannelPatterns = {'CamB_ch0'}

# psf path
psf_rt = rt
PSFFullpaths = [psf_rt + 'PSF/488_2_c.tif']

# RL method
RLMethod = 'omw'
# wiener filter parameter
# alpha parameter should be adjusted based on SNR and data quality.
# typically 0.002 - 0.01 for SNR ~20; 0.02 - 0.1 or higher for SNR ~7
wienerAlpha = 0.005
# OTF thresholding parameter
OTFCumThresh = 0.9
# true if the PSF is in skew space
skewed = True
# deconvolution result path string (within dataPath)
deconPathstr = 'matlab_decon_omw'

# background to subtract
Background = 100
# number of iterations
DeconIter = 2
# erode the edge after decon for number of pixels.
EdgeErosion = 8
# save as 16bit, if false, save to single
Save16bit = True
# use zarr file as input, if false, use tiff as input
zarrFile = False
# number of cpu cores
cpusPerTask = 24
# use cluster computing for different images
parseCluster = False
# set it to true for large files that cannot be fitted to RAM/GPU, it will
# split the data to chunks for deconvolution
largeFile = False
# use GPU for deconvolution
GPUJob = False
# if true, save intermediate results every 5 iterations.
debug = False
# config file for the master jobs that runs on CPU node
ConfigFile = ''
# config file for the GPU job scheduling on GPU node
GPUConfigFile = ''
# if true, use Matlab runtime (for the situation without matlab license)
mccMode = True

# Step 3.2: run the deconvolution with given parameters.
# the results will be saved in matlab_decon under the dataPaths.
# the next step is deskew/rotate (if in skewed space for x-stage scan) or
# rotate (if objective scan) or other processings.

# result folder:
# {destPath}/PetaKit5D_demo_cell_image_dataset/matlab_decon_omw/

XR_decon_data_wrapper(dataPaths, resultDirName=deconPathstr, xyPixelSize=xyPixelSize,
                      dz=dz, reverse=Reverse, channelPatterns=ChannelPatterns, psfFullpaths=PSFFullpaths,
                      dzPSF=dzPSF, parseSettingFile=parseSettingFile, RLMethod=RLMethod,
                      wienerAlpha=wienerAlpha, OTFCumThresh=OTFCumThresh, skewed=skewed,
                      background=Background, deconIter=DeconIter, edgeErosion=EdgeErosion, 
                      save16bit=Save16bit, zarrFile=zarrFile, parseCluster=parseCluster, 
                      largeFile=largeFile, GPUJob=GPUJob, debug=debug, cpusPerTask=cpusPerTask, 
                      configFile=ConfigFile, GPUConfigFile=GPUConfigFile, mccMode=mccMode)

## Run a skewed space stitch on our demo dataset

In [ ]:
from PyPetaKit5D import XR_matlab_stitching_wrapper

# Set your data path to our demo dataset
dataPaths = ["~/Downloads/PetaKit5D_demo_cell_image_dataset/"]

# image list path: csv file
# if not available, run stitch_generate_imagelist_from_encoder(dataPath, dz)
ImageListFullpath = [dataPath+"ImageList_from_encoder.csv"]

# stitch in DS space
DS = False
# stitch in Deskew/rotated space, if both DS and DSR false, stitch in skewed space
DSR = False

# skew angle
SkewAngle = 32.45

# scan direction. 
Reverse = True

# resolution [xyPixelsize, dz]
xyPixelSize = 0.108
dz = 0.30

# check the setting files to see if a tile is flipped (bidirectional scan)
parseSettingFile = False

# axis order
axisOrder = '-x,y,z'

# stitch blending method, 'none': no blending, 'feather': feather blending
BlendMethod = 'feather'

# stitch dir string, inside dataPath
stitchResultDir = 'matlab_stitch'

# cross correlation registration, if false, directly stitch by the coordinates.
xcorrShift = True

# if true, only stitch first time point
onlyFirstTP = False

# stitch pipeline, no need to change, zarr pipeline is the mostly used one.
stitchPipeline = 'zarr'

# xcorr registration on the primary channel, and other channels uses the
# registration informaion from the primary channel
PrimaryCh = 'CamB_ch0'

# channels to stitch
ChannelPatterns = ['CamA_ch0', 'CamB_ch0']

# if true, save result as 16bit, if false, save as single
Save16bit = True

#########################################################################
# parameters below are used in special cases or for advanced fine-tuning

# resample data, if empty, stitch in original resolution. If a 1X3 array,
# resample data and then stitch, mostly used for initial inspection
resample = []

# resample type: isotropic, xy_isotropic; effective when 'resample' is
# not set. Keep it as 'isotropic'. 
resampleType = 'isotropic'

# max allowed shift (in pixel) in xy axes between neighboring tiles in xcorr registration. 
xyMaxOffset = 150

# max allowed shift in z (in pixel) axis between neighboring tiles in xcorr registration. 
zMaxOffset = 150

# downsampling factors for overlap regions to calculate xcorr in xcorr registration. 
# with larger downsamplinf factors, the faster the xcorr computing, but
# the lower accuracy will be. 
xcorrDownsample = [2, 2, 2]

# crop input tile before any processing, empty (no crop) or a 1X6 array [ymin, xmin, zmin, ymax, xmax, zmax]
InputBbox = []

# crop input tile after processing, empty (no crop) or a 1X6 array [ymin, xmin, zmin, ymax, xmax, zmax]
tileOutBbox = []

# chunk size in zarr
blockSize = [256, 256, 256]

# Currently, flat field correction is disabled in the demo
processFunPath = ''

# tile offset: counts add to the image, used when the image background is
# 0, to differentiate between image background and empty space. 
TileOffset = 0

# use slurm cluster if true, otherwise use the local machine (master job)
parseCluster = False
# use master job for task computing or not. 
masterCompute = True
# configuration file for job submission
configFile = ''
# if true, use Matlab runtime (for the situation without matlab license)
mccMode = False


# Step 2: run the stitching with given parameters. 
# the stitched results will be the zarr files in 'matlab_stitch_xcorr_feather_skewed_zarr'
# They are in skewed space, the next step is deconvolution/deskew rotation,
# or just deskew rotation. 


XR_matlab_stitching_wrapper(dataPaths, ImageListFullpath,
    DS=DS, DSR=DSR, skewAngle=SkewAngle, reverse=Reverse, axisOrder=axisOrder,
    xcorrShift=xcorrShift, channelPatterns=ChannelPatterns, primaryCh=PrimaryCh,
    blockSize=blockSize, tileOffset=TileOffset, resampleType=resampleType,
    resampleFactor=resample, xyPixelSize=xyPixelSize, dz=dz, 
    blendMethod=BlendMethod, resultDirName=stitchResultDir, xyMaxOffset=xyMaxOffset,
    zMaxOffset=zMaxOffset, xcorrDownsample=xcorrDownsample, inputBbox=InputBbox,
    tileOutBbox=tileOutBbox, parseSettingFile=parseSettingFile, processFunPath=processFunPath,
    save16bit=Save16bit, parseCluster=parseCluster, masterCompute=masterCompute,
    configFile=configFile, mccMode=mccMode)

## Run a resample on our demo dataset

In [ ]:
from PyPetaKit5D import XR_resample_dataset

# Set your data path to our demo dataset
dataPaths = ['~/Downloads/PetaKit5D_demo_cell_image_dataset/']

# resampling factor: order yxz, downsample 2x if it is 2. 
resampleFactor = [2, 2, 2]

# result path string (within dataPath)
resultDirName = 'resampled'
# channel patterns to map the files for processing
channelPatterns = ['CamA_ch0', 'CamB_ch0']
# interploation method: linear, cubic, nearest
interpMethod = 'linear'
# save as 16 bit if true
save16bit = True
# true if input is in zarr format
zarrFile = False
# true for large files that the processing cannot fit into memory
largeZarr = False
# save output as zarr if true
saveZarr = False
# block size to define the zarr chunk size, only needed for zarr file
blockSize = [256, 256, 256]
# batch size to define the size for each task, only needed for large zarr file
batchSize = [512, 512, 512]
# border buffer size for the resampling, only needed for large zarr file
borderSize = [5, 5, 5]

# use cluster computing for different images
parseCluster = False
# master compute
masterCompute = True
# number of cpu cores
cpusPerTask = 4
# if true, use Matlab runtime (for the situation without matlab license)
mccMode = True
# config file for the master jobs that runs on CPU node
configFile = ''

XR_resample_dataset(dataPaths, resampleFactor, resultDirName=resultDirName,
    channelPatterns=channelPatterns, interpMethod=interpMethod, save16bit=save16bit,
    zarrFile=zarrFile, largeZarr=largeZarr, saveZarr=saveZarr, blockSize=blockSize,
    batchSize=batchSize, borderSize=borderSize, parseCluster=parseCluster,
    masterCompute=masterCompute, cpusPerTask=cpusPerTask, mccMode=mccMode,
    configFile=configFile);

# Run a crop on our demo dataset

In [ ]:
from PyPetaKit5D import XR_crop_dataset

# Set your data path to our demo dataset
dataPaths = ['~/Downloads/PetaKit5D_demo_cell_image_dataset/']

inputBbox = [1, 1, 1, 100, 100, 100]

XR_crop_dataset(dataPaths, inputBbox)

## Create MIPs of our demo dataset

In [ ]:
from PyPetaKit5D import XR_MIP_wrapper

# Set your data path to our demo dataset
dataPaths = ['~/Downloads/PetaKit5D_demo_cell_image_dataset/']

# result path string (within dataPath)
resultDirName = 'MIPs'
# axes for the MIP: xyz
axis = [1, 1, 1]
# channel patterns to map the files for processing
channelPatterns = ['CamA_ch0', 'CamB_ch0']
# true if input is in zarr format
zarrFile = False
# true for large files that the processing cannot fit into memory
largeZarr = False
# batch size for a task, only needed for zarr file
batchSize = [1024, 1024, 1024]
# save as 16 bit if true
save16bit = True

# use cluster computing for different images
parseCluster = False
# master compute
masterCompute = True
# number of cpu cores
cpusPerTask = 4
# if true, use Matlab runtime (for the situation without matlab license)
mccMode = True
# config file for the master jobs that runs on CPU node
configFile = ''

XR_MIP_wrapper(dataPaths, resultDirName=resultDirName, axis=axis, channelPatterns=channelPatterns,
    zarrFile=zarrFile, largeZarr=largeZarr, batchSize=batchSize, save16bit=save16bit,
    parseCluster=parseCluster, masterCompute=masterCompute, cpusPerTask=cpusPerTask,
    mccMode=mccMode, configFile=configFile);

## Run an FFT Analysis on our demo dataset

In [ ]:
# Note: We must first run a Decon and DS/DSR before the FFT Analysis
from PyPetaKit5D import XR_decon_data_wrapper
from PyPetaKit5D import XR_deskew_rotate_data_wrapper
from PyPetaKit5D import XR_fftSpectrumComputingWrapper
import math

# Run the decon
# Set your data path to our demo dataset
dataPath = '~/Downloads/PetaKit5D_demo_cell_image_dataset/'
# root path
rt = dataPath
# data path for data to be deconvolved, also support for multiple data folders
dataPaths = [dataPath]

# xy pixel size in um
xyPixelSize = 0.108
# z step size
dz = 0.3
# scan direction
Reverse = True
# psf z step size (we assume xyPixelSize also apply to psf)
dzPSF = 0.3

# if true, check whether image is flipped in z using the setting files
parseSettingFile = False

# channel patterns for the channels, the channel patterns should map the
# order of PSF filenames.
channelPatterns = ['CamB_ch0']

# psf path
psf_rt = rt            
psfFullpaths = [psf_rt+'/PSF/488_2_c.tif']            

# RL method
RLMethod = 'omw'
# wiener filter parameter
# alpha parameter should be adjusted based on SNR and data quality.
# typically 0.002 - 0.01 for SNR ~20; 0.02 - 0.1 or higher for SNR ~7
wienerAlpha = 0.005
# OTF thresholding parameter
OTFCumThresh = 0.9
# true if the PSF is in skew space
skewed = True
# deconvolution result path string (within dataPath)
resultDirName = 'matlab_decon_omw'

# background to subtract
background = 100
# number of iterations
deconIter = 2
# erode the edge after decon for number of pixels.
edgeErosion = 0
# save as 16bit; if false, save to single
save16bit = True
# use zarr file as input; if false, use tiff as input
zarrFile = False
# save output as zarr file; if false, save as tiff
saveZarr = False
# number of cpu cores
cpusPerTask = 24
# use cluster computing for different images
parseCluster = False
# set it to true for large files that cannot be fitted to RAM/GPU, it will
# split the data to chunks for deconvolution
largeFile = False
# use GPU for deconvolution
GPUJob = False
# if true, save intermediate results every 5 iterations.
debug = False
# config file for the master jobs that runs on CPU node
configFile = ''
# config file for the GPU job scheduling on GPU node
GPUConfigFile = ''
# if true, use Matlab runtime (for the situation without matlab license)
mccMode = True

# result folder:
# {destPath}/PetaKit5D_demo_cell_image_dataset/matlab_decon_omw/

XR_decon_data_wrapper(dataPaths, resultDirName=resultDirName, xyPixelSize=xyPixelSize,
    dz=dz, channelPatterns=channelPatterns, psfFullpaths=psfFullpaths,
    dzPSF=dzPSF, parseSettingFile=parseSettingFile, RLMethod=RLMethod,
    wienerAlpha=wienerAlpha, OTFCumThresh=OTFCumThresh, skewed=skewed,
    background=background, deconIter=deconIter, edgeErosion=edgeErosion, 
    save16bit=save16bit, zarrFile=zarrFile, saveZarr=saveZarr, parseCluster=parseCluster,
    largeFile=largeFile, GPUJob=GPUJob, debug=debug, cpusPerTask=cpusPerTask,
    configFile=configFile, GPUConfigFile=GPUConfigFile, mccMode=mccMode)

# Run deskew/rotation

# result folder:
# {destPath}/PetaKit5D_demo_cell_image_dataset/matlab_decon_omw/DS_for_useful_tools/
# {destPath}/PetaKit5D_demo_cell_image_dataset/matlab_decon_omw/DSR_for_useful_tools/

dataPaths = [dataPath+'/matlab_decon_omw/']

# deskew result path string (within dataPath)
DSDirName = 'DS_for_useful_tools/'
# deskew/rotate result path string (within dataPath)
DSRDirName = 'DSR_for_useful_tools/'

# run deskew if true
deskew = True
# run rotation if true
rotate = True
# use combined processing if true. Note: if you only need deskew without
# rotation, set both Rotate and DSRCombined as false.
DSRCombined = False
# xy pixel size
xyPixelSize = 0.108
# z scan step size
dz = 0.3
# Skew angle
skewAngle = 32.45
# if true, objective scan; otherwise, sample scan
objectiveScan = False
# scan direction
reverse = True
# channel patterns to map the files for processing
channelPatterns = ['CamA', 'CamB']

# flat field related parameters
# disable flat field for this demo
FFCorrection = False
# lower bound cap for flat field
lowerLimit = 0.4
# constant offset after flat field correction
constOffset = 1
# flat field image paths
FFImagePaths = [
    dataPath + 'FF/averaged/ff_CamA_ch0_CAM1_stack0000_488nm_0000000msec_0096911533msecAbs_000x_000y_000z_0017t.tif',
    dataPath + 'FF/averaged/ff_CamB_ch0_CAM1_stack0000_488nm_0000000msec_0096911533msecAbs_000x_000y_000z_0017t.tif']

# background image paths
backgroundPaths = [dataPath + 'FF/KorraFusions/AVG_DF400_CamA_10ms.tif',
                   dataPath + 'FF/KorraFusions/AVG_DF400_CamB_10ms.tif']

# if true, use large scale processing pipeline (split, process, and then merge)
largeFile = False
# true if input is in zarr format
zarrFile = False
# save output as zarr if true
saveZarr = True
# block size to save the result 
blockSize = [256, 256, 256]
# save output as uint16 if true
save16bit = True

# use slurm cluster if true, otherwise use the local machine (master job)
parseCluster = False
# use master job for task computing or not. 
masterCompute = True
# configuration file for job submission
configFile = ''
# if true, use Matlab runtime (for the situation without matlab license)
mccMode = True

XR_deskew_rotate_data_wrapper(dataPaths, DSDirName=DSDirName, DSRDirName=DSRDirName,
    deskew=deskew, rotate=rotate, DSRCombined=DSRCombined, xyPixelSize=xyPixelSize,
    dz=dz, skewAngle=skewAngle, objectiveScan=objectiveScan, reverse=reverse,
    channelPatterns=channelPatterns, FFCorrection=FFCorrection, lowerLimit=lowerLimit,
    constOffset=constOffset, FFImagePaths=FFImagePaths, backgroundPaths=backgroundPaths,
    largeFile=largeFile, zarrFile=zarrFile, saveZarr=saveZarr, blockSize=blockSize,
    save16bit=save16bit, parseCluster=parseCluster, masterCompute=masterCompute,
    configFile=configFile, mccMode=mccMode)

# FFT analysis
# compute fourier power spectrum of the given images, by default only the central 
# slides are saved with gamma 0.5. If set save3DStack as true, also save 3D 
# stack with log10 applied.

dataPaths = [dataPath+'/matlab_decon_omw/DS_for_useful_tools/']

# result path string (within dataPaths)
resultDirName = 'FFT'
# xy pixel size
xyPixelSize = 0.108
# z scan step size (the input data is deskewed, so the z step size is
# original dz * sind(theta). 
dz = 0.3 * math.sin(math.radians(32.45))
# true if input is in zarr format
zarrFile = True
# output size for FFT spectrum
outSize = [1001, 1001, 1001]
# output voxel size, isotropic, here we use 0.1 medium wavelength
outPixelSize = 0.1 * 0.488 / 1.33
# save 3D spectrum if true.
save3DStack = False

# use cluster computing for different images
parseCluster = False
# master compute
masterCompute = True
# number of cpu cores
cpusPerTask = 24
# if true, use Matlab runtime (for the situation without matlab license)
mccMode = True
# config file for the master jobs that runs on CPU node
configFile = ''

XR_fftSpectrumComputingWrapper(dataPaths, resultDirName=resultDirName,
    xyPixelSize=xyPixelSize, dz=dz, zarrFile=zarrFile, outSize=outSize,
    outPixelSize=outPixelSize, save3DStack=save3DStack, parseCluster=parseCluster,
    masterCompute=masterCompute, cpusPerTask=cpusPerTask, mccMode=mccMode,
    configFile=configFile);

# Run a PSF Analysis on our demo dataset

In [ ]:
# Note: This demo will not end until you close the final graph
from PyPetaKit5D import XR_psf_analysis_wrapper
import os
from IPython.display import Image, display

# Set your data path to our demo dataset
dataPath = '~/Downloads/PetaKit5D_demo_cell_image_dataset/'
dataPaths = [dataPath+'/PSF/']

# xy pixel size
xyPixelSize = 0.108
# z step size
dz = 0.3
# Skew angle
skewAngle = 32.45
# channel patterns to map the files for processing
channelPatterns = ['488']
# channel wavelength
channels = [488]
# deskew PSF if true. This is for PSF in the skewed space
deskew = True
# if true, objective scan; otherwise, sample scan
objectiveScan = False

# Richards & Wolf theoretical PSF path for corresponding channels
RWFn = [dataPath+'/PSF/RW_PSFs/PSF_RW_515em_128_128_101_100nmSteps.tif']
# source description
sourceStr = 'Demo PSF'

# use cluster computing for different images
parseCluster = False
# master compute
masterCompute = True
# number of cpu cores
cpusPerTask = 24
# if true, use Matlab runtime (for the situation without matlab license)
mccMode = True
# config file for the master jobs that runs on CPU node
configFile = ''

XR_psf_analysis_wrapper(dataPaths, xyPixelSize=xyPixelSize, dz=dz,
    skewAngle=skewAngle, channelPatterns=channelPatterns, channels=channels,
    deskew=deskew, objectiveScan=objectiveScan, sourceStr=sourceStr,
    RWFn=RWFn, parseCluster=parseCluster, masterCompute=masterCompute,
    cpusPerTask=cpusPerTask, mccMode=mccMode, configFile=configFile, visible=False);

# Display the graphs
im1 = Image(filename = f"{os.path.expanduser(dataPaths[0])}/DS/PSFAnalysis/comp_488_2_c.png")
im2 = Image(filename = f"{os.path.expanduser(dataPaths[0])}/DS/PSFAnalysis/wT_488_2_c.png")
display(im1,im2)

## Run an FSC Analysis on our demo dataset

In [ ]:
# Note: We must first run a Decon before the FSC Analysis
from PyPetaKit5D import XR_decon_data_wrapper
from PyPetaKit5D import XR_FSC_analysis_wrapper
import math

# Run Decon with different number of iterations

# Set your data path to our demo dataset
# data path for data to be deconvolved, also support for multiple data folders
dataPath = '~/Downloads/PetaKit5D_demo_cell_image_dataset/'
dataPaths = [dataPath]

# root path
rt = dataPath

# xy pixel size in um
xyPixelSize = 0.108
# z step size
dz = 0.3
# psf z step size (we assume xyPixelSize also apply to psf)
dzPSF = 0.3

# if true, check whether image is flipped in z using the setting files
parseSettingFile = False

# channel patterns for the channels, the channel patterns should map the
# order of PSF filenames.
# we only deconvolve one tile for FSC computing and visualization
channelPatterns = ['Scan_Iter_0000_0000_CamB_ch0_CAM1_stack0000_488nm_0000000msec_0106060251msecAbs_000x_003y_000z_0000t']  

# psf path
psf_rt = rt            
PSFFullpaths = [psf_rt+'/PSF/488_2_c.tif']            

# RL method
RLMethod = 'omw'
# wiener filter parameter
# alpha parameter should be adjusted based on SNR and data quality.
# typically 0.002 - 0.01 for SNR ~20; 0.02 - 0.1 or higher for SNR ~7
# here we use larger alpha for more stable results with more iterations in
# the debug mode
wienerAlpha = 0.05
# OTF thresholding parameter
OTFCumThresh = 0.9
# true if the PSF is in skew space
skewed = True
# deconvolution result path string (within dataPath)
resultDirName = 'matlab_decon_omw_debug'

# background to subtract
background = 100
# number of iterations
deconIter = 20
# erode the edge after decon for number of pixels.
edgeErosion = 0
# save as 16bit; if false, save to single
save16bit = True
# use zarr file as input; if false, use tiff as input
zarrFile = False
# save output as zarr file; if false,s ave as tiff
saveZarr = False
# number of cpu cores
cpusPerTask = 24
# use cluster computing for different images
parseCluster = False
# set it to true for large files that cannot be fitted to RAM/GPU, it will
# split the data to chunks for deconvolution
largeFile = False
# use GPU for deconvolution
GPUJob = False
# save Step 
saveStep = 2
# if true, save intermediate results every #saveStep iterations.
debug = True
# config file for the master jobs that runs on CPU node
configFile = ''
# config file for the GPU job scheduling on GPU node
GPUConfigFile = ''
# if true, use Matlab runtime (for the situation without matlab license)
mccMode = True

# result folder:
# {destPath}/PetaKit5D_demo_cell_image_dataset/matlab_decon_omw/

XR_decon_data_wrapper(dataPaths, resultDirName=resultDirName, xyPixelSize=xyPixelSize,
    dz=dz, channelPatterns=channelPatterns, psfFullpaths=psfFullpaths,
    dzPSF=dzPSF, parseSettingFile=parseSettingFile, RLMethod=RLMethod,
    wienerAlpha=wienerAlpha, OTFCumThresh=OTFCumThresh, skewed=skewed,
    background=background, deconIter=deconIter, edgeErosion=edgeErosion, 
    save16bit=save16bit, zarrFile=zarrFile, saveZarr=saveZarr, 
    parseCluster=parseCluster, largeFile=largeFile, GPUJob=GPUJob,
    saveStep=saveStep, debug=debug, cpusPerTask=cpusPerTask,
    configFile=configFile, GPUConfigFile=GPUConfigFile, mccMode=mccMode);


# FSC analysis
# perform fourier shell correlation (FSC) analysis for images in one or
# multiple folders. Here we run FSC analysis for the deconvolution with different 
# iterations to demonstrate how to use FSC to determine the optimal number
# of iterations.  

# result folder:
# {destPath}/PetaKit5D_demo_cell_image_dataset/matlab_decon_omw_debug/Scan_Iter_0000_0000_CamB_ch0_CAM1_stack0000_488nm_0000000msec_0106060251msecAbs_000x_003y_000z_0000t_debug/FSCs
# {destPath}/PetaKit5D_demo_cell_image_dataset/matlab_decon_omw_debug/Scan_Iter_0000_0000_CamB_ch0_CAM1_stack0000_488nm_0000000msec_0106060251msecAbs_000x_003y_000z_0000t_debug/figures

dataPaths = [dataPath+'/matlab_decon_omw_debug/Scan_Iter_0000_0000_CamB_ch0_CAM1_stack0000_488nm_0000000msec_0106060251msecAbs_000x_003y_000z_0000t_debug/']

# result path string (within dataPaths)
resultDirName = 'FSCs'
# xy pixel size
xyPixelSize = 0.108
# z scan step size (the input data is deskewed, so the z step size is
# original dz * sind(theta). 
dz = 0.3 * math.sin(math.radians(32.45))
# radial interval for the calculation of correlation
dr = 5
# angular interval for the calculation of correlation
dtheta = math.pi / 6
# resolution thresholding method: one-bit, half-bit, or fixed 
resThreshMethod = 'one-bit'
# resolution threshold (only for fixed method)
resThresh = 0.2
# half size of region for FSC (typically leave some extra space for border)
# must be the same across all three dimensions and better to be odd number
halfSize = [201, 201, 201]
# input bounding box to define the region for FSC computing. If it is
# empty, use the central region for the computing. 
inputBbox = []
# channel patterns to map the files for processing
channelPatterns = ['Iter']
# channel wavelength (only for figure naming purpose)
channels = [488]
# suffix (only for figure naming purpose)
suffix = 'decon'
# iteration interval for figure plotting
iterInterval = 2

# use cluster computing for different images
parseCluster = False
# master compute
masterCompute = True
# number of cpu cores
cpusPerTask = 24
# if true, use Matlab runtime (for the situation without matlab license)
mccMode = True
# config file for the master jobs that runs on CPU node
configFile = ''

XR_FSC_analysis_wrapper(dataPaths, resultDirName=resultDirName, xyPixelSize=xyPixelSize,
    dz=dz, dr=dr, dtheta=dtheta, resThreshMethod=resThreshMethod,
    resThresh=resThresh, halfSize=halfSize, inputBbox=inputBbox,
    channelPatterns=channelPatterns, channels=channels, suffix=suffix,
    iterInterval=iterInterval, parseCluster=parseCluster, masterCompute=masterCompute,
    cpusPerTask=cpusPerTask, mccMode=mccMode, configFile=configFile)

## Convert tiff files to zarr files and zarr files to tiff files using our demo dataset

In [ ]:
from PyPetaKit5D import XR_tiffToZarr_wrapper
from PyPetaKit5D import XR_zarrToTiff_wrapper

# First we will convert our tiff files to zarr files

# result folder:
# {destPath}/PetaKit5D_demo_cell_image_dataset/tiff_to_zarr/

# Set your data path to our demo dataset
dataPath = '~/Downloads/PetaKit5D_demo_cell_image_dataset/'
dataPaths = [dataPath]

# result path string
resultDirName = 'tiff_to_zarr'
# channel patterns to map the files for processing
channelPatterns = ['CamA_ch0', 'CamB_ch0']
# use cluster computing for different images
# bounding box to crop the files in the conversion: [ymin, xmin, zmin, ymax, xmax, zmax]
inputBbox = []

parseCluster = False
# master compute
masterCompute = True
# number of cpu cores
cpusPerTask = 2
# if true, use Matlab runtime (for the situation without matlab license)
mccMode = False
# config file for the master jobs that runs on CPU node
configFile = ''

XR_tiffToZarr_wrapper(dataPaths, resultDirName=resultDirName, channelPatterns=channelPatterns,
    inputBbox=inputBbox, parseCluster=parseCluster, masterCompute=masterCompute,
    cpusPerTask=cpusPerTask, mccMode=mccMode, configFile=configFile)

# Now we will convert the zarr files we just made back into tiff files

# result folder:
# {destPath}/PetaKit5D_demo_cell_image_dataset/tiff_to_zarr/tiffs/

dataPaths = [dataPath+'/'+resultDirName]

# result path string (within dataPaths)
resultDirName = 'tiffs'
# channel patterns to map the files for processing
channelPatterns = ['CamA_ch0', 'CamB_ch0']
# use cluster computing for different images
parseCluster = False
# master compute
masterCompute = True
# number of cpu cores
cpusPerTask = 2
# if true, use Matlab runtime (for the situation without matlab license)
mccMode = True
# config file for the master jobs that runs on CPU node
configFile = ''

XR_zarrToTiff_wrapper(dataPaths, resultDirName=resultDirName, channelPatterns=channelPatterns,
    parseCluster=parseCluster, masterCompute=masterCompute, cpusPerTask=cpusPerTask,
    mccMode=mccMode, configFile=configFile)

## Create an imaris file from our demo dataset

In [ ]:
from PyPetaKit5D import XR_imaris_conversion_data_wrapper

# Set your data path to our demo dataset
dataPaths = ['~/Downloads/PetaKit5D_demo_cell_image_dataset/']

# result path string (within dataPaths)
resultDirName = 'imaris'
# since the demo dataset has several tiles, we only convert the time series
# for one tile. 
channelPatterns = ['CamA_ch0', 'CamB_ch0']
# voxel sizes for the data: y, x, z
pixelSizes = [0.108, 0.108, 0.108]
# use zarr file as input if true
zarrFile = False
# block size for the imaris file
blockSize = [64, 64, 64]

parseCluster = False
# master compute
masterCompute = True
# number of cpu cores
cpusPerTask = 24
# if true, use Matlab runtime (for the situation without matlab license)
mccMode = False
# config file for the master jobs that runs on CPU node
configFile = ''

XR_imaris_conversion_data_wrapper(dataPaths, channelPatterns=channelPatterns,
    pixelSizes=pixelSizes, zarrFile=zarrFile, blockSize=blockSize,
    parseCluster=parseCluster, masterCompute=masterCompute, cpusPerTask=cpusPerTask,
    mccMode=mccMode, configFile=configFile);